# Market Data Exploration with OKX Client Gateway

This notebook showcases the market data capabilities of `okx-client-gw-py`, focusing on:
- Real-time market data retrieval
- Historical data analysis
- Order book analysis
- WebSocket streaming for real-time updates

**Note**: All examples in this notebook use public endpoints and do not require authentication.

## Setup and Imports

In [ ]:
import asyncio
from datetime import datetime, timedelta
from decimal import Decimal

# Core imports
from okx_client_gw import OkxHttpClient, OkxConfig

# Services
from okx_client_gw.application.services import (
    MarketDataService,
    InstrumentService,
    StreamingService,
    MultiChannelStreamingService,
    PublicDataService,
)

# Domain models and enums
from okx_client_gw.domain.enums import InstType

# Configuration
config = OkxConfig(demo=True)

## Part 1: Instrument Discovery

### Explore Available Markets

In [ ]:
async def explore_markets():
    """Get an overview of all available markets."""
    async with OkxHttpClient(config=config) as client:
        service = InstrumentService(client)
        
        market_types = [
            (InstType.SPOT, "Spot"),
            (InstType.SWAP, "Perpetual Swaps"),
            (InstType.FUTURES, "Futures"),
            (InstType.OPTION, "Options"),
        ]
        
        print("OKX Market Overview:")
        print("=" * 50)
        
        totals = {}
        for inst_type, name in market_types:
            try:
                instruments = await service.get_instruments(inst_type)
                totals[name] = len(instruments)
                print(f"{name:20} {len(instruments):>6} instruments")
            except Exception as e:
                print(f"{name:20} Error: {e}")
        
        print("=" * 50)
        print(f"{'Total':20} {sum(totals.values()):>6} instruments")
        
        return totals

market_overview = await explore_markets()

### Analyze Spot Market Structure

In [ ]:
async def analyze_spot_structure():
    """Analyze the structure of spot markets."""
    async with OkxHttpClient(config=config) as client:
        service = InstrumentService(client)
        instruments = await service.get_instruments(InstType.SPOT)
        
        # Group by quote currency
        quote_currencies = {}
        for inst in instruments:
            quote = inst.quote_ccy
            if quote not in quote_currencies:
                quote_currencies[quote] = []
            quote_currencies[quote].append(inst)
        
        print("Spot Markets by Quote Currency:")
        print("-" * 40)
        for quote, pairs in sorted(quote_currencies.items(), key=lambda x: -len(x[1])):
            print(f"  {quote:8} {len(pairs):>4} pairs")
        
        # Group by base currency (top assets)
        base_currencies = {}
        for inst in instruments:
            base = inst.base_ccy
            if base not in base_currencies:
                base_currencies[base] = []
            base_currencies[base].append(inst)
        
        print("\nTop 10 Assets by Trading Pairs:")
        print("-" * 40)
        sorted_bases = sorted(base_currencies.items(), key=lambda x: -len(x[1]))[:10]
        for base, pairs in sorted_bases:
            pair_list = ", ".join([p.quote_ccy for p in pairs])
            print(f"  {base:8} {len(pairs):>2} pairs: {pair_list}")
        
        return quote_currencies, base_currencies

quotes, bases = await analyze_spot_structure()

### Find Specific Instruments

In [ ]:
async def find_instruments(search_term: str):
    """Find instruments matching a search term."""
    async with OkxHttpClient(config=config) as client:
        service = InstrumentService(client)
        
        print(f"Searching for '{search_term}'...\n")
        
        results = []
        for inst_type in [InstType.SPOT, InstType.SWAP, InstType.FUTURES]:
            instruments = await service.get_instruments(inst_type)
            matches = [i for i in instruments if search_term.upper() in i.inst_id.upper()]
            results.extend([(inst_type, i) for i in matches])
        
        if results:
            print(f"Found {len(results)} instruments:")
            print(f"{'Type':<10} {'Instrument':<20} {'Min Size':<12} {'Tick Size':<12}")
            print("-" * 56)
            for inst_type, inst in results[:15]:
                print(f"{inst_type.value:<10} {inst.inst_id:<20} {str(inst.min_sz):<12} {str(inst.tick_sz):<12}")
            if len(results) > 15:
                print(f"... and {len(results) - 15} more")
        else:
            print("No instruments found")
        
        return results

# Search examples
btc_results = await find_instruments("BTC-USDT")

## Part 2: Real-Time Market Data

### Market Snapshot

In [ ]:
async def market_snapshot(inst_id: str):
    """Get a comprehensive market snapshot for an instrument."""
    async with OkxHttpClient(config=config) as client:
        market_service = MarketDataService(client)
        instrument_service = InstrumentService(client)
        
        # Get instrument info
        inst_type = InstType.SPOT if "-" in inst_id and inst_id.count("-") == 1 else InstType.SWAP
        instrument = await instrument_service.get_instrument(inst_type, inst_id)
        
        # Get ticker
        ticker = await market_service.get_ticker(inst_id)
        
        # Get order book
        orderbook = await market_service.get_orderbook(inst_id, depth=5)
        
        # Get recent trades
        trades = await market_service.get_trades(inst_id, limit=10)
        
        print(f"Market Snapshot: {inst_id}")
        print("=" * 60)
        
        print(f"\n📊 Instrument Info:")
        if instrument:
            print(f"  Type:        {instrument.inst_type.value}")
            print(f"  Base:        {instrument.base_ccy}")
            print(f"  Quote:       {instrument.quote_ccy}")
            print(f"  Min Size:    {instrument.min_sz}")
            print(f"  Tick Size:   {instrument.tick_sz}")
        
        print(f"\n💰 Price Info:")
        print(f"  Last:        ${float(ticker.last):,.2f}")
        print(f"  24h High:    ${float(ticker.high_24h):,.2f}")
        print(f"  24h Low:     ${float(ticker.low_24h):,.2f}")
        print(f"  24h Change:  {float(ticker.change_24h) * 100:+.2f}%")
        print(f"  24h Volume:  {float(ticker.vol_24h):,.2f}")
        
        print(f"\n📖 Order Book (top 5):")
        print(f"  Spread:      ${orderbook.spread:.2f} ({orderbook.spread / float(ticker.last) * 100:.4f}%)")
        print(f"  Mid Price:   ${orderbook.mid_price:,.2f}")
        print(f"\n  {'Asks':<20} {'Bids':>20}")
        for i in range(min(5, len(orderbook.asks), len(orderbook.bids))):
            ask = orderbook.asks[i]
            bid = orderbook.bids[i]
            print(f"  {float(ask.sz):.4f} @ ${float(ask.px):,.2f}  |  ${float(bid.px):,.2f} @ {float(bid.sz):.4f}")
        
        print(f"\n🔄 Recent Trades:")
        for trade in trades[:5]:
            side = "↑" if trade.side == "buy" else "↓"
            print(f"  {trade.ts.strftime('%H:%M:%S')} {side} {float(trade.sz):.6f} @ ${float(trade.px):,.2f}")
        
        return ticker, orderbook, trades

snapshot = await market_snapshot("BTC-USDT")

### Order Book Depth Analysis

In [ ]:
async def analyze_orderbook(inst_id: str, depth: int = 20):
    """Analyze order book depth and liquidity."""
    async with OkxHttpClient(config=config) as client:
        service = MarketDataService(client)
        ticker = await service.get_ticker(inst_id)
        orderbook = await service.get_orderbook(inst_id, depth=depth)
        
        mid_price = float(ticker.last)
        
        # Calculate cumulative depth
        bid_depth = sum(float(l.sz) for l in orderbook.bids)
        ask_depth = sum(float(l.sz) for l in orderbook.asks)
        
        # Calculate depth at different price levels
        levels = [0.1, 0.5, 1.0, 2.0]  # Percentage from mid
        
        print(f"Order Book Analysis: {inst_id}")
        print("=" * 60)
        print(f"Mid Price: ${mid_price:,.2f}")
        print(f"Spread: ${orderbook.spread:.2f} ({orderbook.spread/mid_price*100:.4f}%)")
        print()
        
        print("Liquidity Summary:")
        print(f"  Total Bid Depth: {bid_depth:,.4f}")
        print(f"  Total Ask Depth: {ask_depth:,.4f}")
        print(f"  Imbalance:       {(bid_depth - ask_depth) / (bid_depth + ask_depth) * 100:+.2f}%")
        print()
        
        print("Depth by Price Level:")
        print(f"{'Level':>8} {'Bid Depth':>15} {'Ask Depth':>15} {'Imbalance':>12}")
        print("-" * 52)
        
        for pct in levels:
            lower_bound = mid_price * (1 - pct/100)
            upper_bound = mid_price * (1 + pct/100)
            
            bid_at_level = sum(float(l.sz) for l in orderbook.bids if float(l.px) >= lower_bound)
            ask_at_level = sum(float(l.sz) for l in orderbook.asks if float(l.px) <= upper_bound)
            
            total = bid_at_level + ask_at_level
            imbalance = (bid_at_level - ask_at_level) / total * 100 if total > 0 else 0
            
            print(f"{pct:>7.1f}% {bid_at_level:>15.4f} {ask_at_level:>15.4f} {imbalance:>+11.2f}%")
        
        return orderbook

ob_analysis = await analyze_orderbook("BTC-USDT")

## Part 3: Historical Data Analysis

### Candlestick Data

In [ ]:
async def get_candle_data(inst_id: str, bar: str = "1H", limit: int = 100):
    """Get and analyze candlestick data."""
    async with OkxHttpClient(config=config) as client:
        service = MarketDataService(client)
        candles = await service.get_candles(inst_id, bar=bar, limit=limit)
        
        if not candles:
            print("No candle data available")
            return None
        
        # Calculate statistics
        closes = [float(c.close) for c in candles]
        volumes = [float(c.vol) for c in candles]
        
        avg_close = sum(closes) / len(closes)
        min_close = min(closes)
        max_close = max(closes)
        avg_volume = sum(volumes) / len(volumes)
        
        # Calculate volatility (simple)
        returns = [(closes[i] - closes[i+1]) / closes[i+1] * 100 for i in range(len(closes)-1)]
        avg_return = sum(returns) / len(returns) if returns else 0
        volatility = (sum((r - avg_return)**2 for r in returns) / len(returns))**0.5 if returns else 0
        
        print(f"Candlestick Analysis: {inst_id} ({bar})")
        print("=" * 60)
        print(f"Period: {candles[-1].ts.strftime('%Y-%m-%d %H:%M')} to {candles[0].ts.strftime('%Y-%m-%d %H:%M')}")
        print(f"Candles: {len(candles)}")
        print()
        
        print("Price Statistics:")
        print(f"  Current:     ${closes[0]:,.2f}")
        print(f"  Average:     ${avg_close:,.2f}")
        print(f"  High:        ${max_close:,.2f}")
        print(f"  Low:         ${min_close:,.2f}")
        print(f"  Range:       ${max_close - min_close:,.2f} ({(max_close - min_close) / avg_close * 100:.2f}%)")
        print()
        
        print("Volume Statistics:")
        print(f"  Average:     {avg_volume:,.2f}")
        print(f"  Latest:      {volumes[0]:,.2f}")
        print()
        
        print("Volatility:")
        print(f"  Per Bar:     {volatility:.4f}%")
        print(f"  Annualized:  {volatility * (365 * 24 / (1 if bar == '1H' else 24 if bar == '1D' else 1))**0.5:.2f}% (approx)")
        
        return candles

candles = await get_candle_data("BTC-USDT", bar="1H", limit=24)

### Historical Price Chart Data

In [ ]:
async def get_historical_data(inst_id: str, bar: str = "1D", limit: int = 30):
    """Get historical data suitable for charting."""
    async with OkxHttpClient(config=config) as client:
        service = MarketDataService(client)
        candles = await service.get_candles(inst_id, bar=bar, limit=limit)
        
        print(f"Historical Data: {inst_id} ({bar}, last {len(candles)} periods)")
        print("=" * 80)
        print(f"{'Date':>12} {'Open':>12} {'High':>12} {'Low':>12} {'Close':>12} {'Volume':>15}")
        print("-" * 80)
        
        for candle in candles[:10]:  # Show last 10
            print(f"{candle.ts.strftime('%Y-%m-%d'):>12} ${float(candle.open):>10,.0f} ${float(candle.high):>10,.0f} ${float(candle.low):>10,.0f} ${float(candle.close):>10,.0f} {float(candle.vol):>15,.2f}")
        
        if len(candles) > 10:
            print(f"... {len(candles) - 10} more rows")
        
        # Simple trend analysis
        first_close = float(candles[-1].close)
        last_close = float(candles[0].close)
        change_pct = (last_close - first_close) / first_close * 100
        
        print()
        print(f"Period Change: {change_pct:+.2f}% (${first_close:,.0f} → ${last_close:,.0f})")
        
        return candles

daily_data = await get_historical_data("BTC-USDT", bar="1D", limit=30)

### Trade History Analysis

In [ ]:
async def analyze_trade_flow(inst_id: str, num_trades: int = 100):
    """Analyze recent trade flow."""
    async with OkxHttpClient(config=config) as client:
        service = MarketDataService(client)
        trades = await service.get_trades(inst_id, limit=num_trades)
        
        if not trades:
            print("No trade data available")
            return None
        
        # Aggregate by side
        buy_trades = [t for t in trades if t.side == "buy"]
        sell_trades = [t for t in trades if t.side == "sell"]
        
        buy_volume = sum(float(t.sz) for t in buy_trades)
        sell_volume = sum(float(t.sz) for t in sell_trades)
        total_volume = buy_volume + sell_volume
        
        buy_value = sum(float(t.sz) * float(t.px) for t in buy_trades)
        sell_value = sum(float(t.sz) * float(t.px) for t in sell_trades)
        
        # Time range
        if trades:
            time_span = (trades[0].ts - trades[-1].ts).total_seconds() / 60
        else:
            time_span = 0
        
        print(f"Trade Flow Analysis: {inst_id}")
        print("=" * 60)
        print(f"Trades: {len(trades)} over {time_span:.1f} minutes")
        print()
        
        print("Volume Breakdown:")
        print(f"  Buy Volume:   {buy_volume:,.6f} ({buy_volume/total_volume*100:.1f}%)")
        print(f"  Sell Volume:  {sell_volume:,.6f} ({sell_volume/total_volume*100:.1f}%)")
        print(f"  Net Flow:     {buy_volume - sell_volume:+,.6f}")
        print()
        
        print("Value Breakdown:")
        print(f"  Buy Value:    ${buy_value:,.2f}")
        print(f"  Sell Value:   ${sell_value:,.2f}")
        print(f"  Net Value:    ${buy_value - sell_value:+,.2f}")
        print()
        
        # Average trade size
        avg_buy_size = buy_volume / len(buy_trades) if buy_trades else 0
        avg_sell_size = sell_volume / len(sell_trades) if sell_trades else 0
        
        print("Average Trade Size:")
        print(f"  Buy:          {avg_buy_size:,.6f}")
        print(f"  Sell:         {avg_sell_size:,.6f}")
        
        return trades

trade_flow = await analyze_trade_flow("BTC-USDT")

## Part 4: Multi-Market Comparison

### Compare Spot vs Swap Prices

In [ ]:
async def compare_spot_swap():
    """Compare spot and perpetual swap prices."""
    async with OkxHttpClient(config=config) as client:
        market_service = MarketDataService(client)
        public_service = PublicDataService(client)
        
        pairs = [
            ("BTC-USDT", "BTC-USDT-SWAP"),
            ("ETH-USDT", "ETH-USDT-SWAP"),
            ("SOL-USDT", "SOL-USDT-SWAP"),
        ]
        
        print("Spot vs Perpetual Swap Comparison:")
        print("=" * 80)
        print(f"{'Asset':<10} {'Spot':>12} {'Swap':>12} {'Basis':>10} {'Basis %':>10} {'Funding':>12}")
        print("-" * 80)
        
        for spot_id, swap_id in pairs:
            try:
                spot_ticker = await market_service.get_ticker(spot_id)
                swap_ticker = await market_service.get_ticker(swap_id)
                funding = await public_service.get_funding_rate(swap_id)
                
                spot_price = float(spot_ticker.last)
                swap_price = float(swap_ticker.last)
                basis = swap_price - spot_price
                basis_pct = basis / spot_price * 100
                funding_pct = float(funding.funding_rate) * 100
                
                asset = spot_id.split("-")[0]
                print(f"{asset:<10} ${spot_price:>10,.2f} ${swap_price:>10,.2f} ${basis:>+9,.2f} {basis_pct:>+9.4f}% {funding_pct:>+11.4f}%")
            except Exception as e:
                print(f"{spot_id}: Error - {e}")
        
        print()
        print("💡 Basis = Swap Price - Spot Price")
        print("   Positive basis (contango) typically aligns with positive funding")
        print("   Negative basis (backwardation) typically aligns with negative funding")

await compare_spot_swap()

### Market Breadth Analysis

In [ ]:
async def market_breadth():
    """Analyze market breadth (advancing vs declining)."""
    async with OkxHttpClient(config=config) as client:
        service = MarketDataService(client)
        tickers = await service.get_tickers(InstType.SPOT)
        
        # Filter for USDT pairs with volume
        usdt_tickers = [t for t in tickers if t.inst_id.endswith("-USDT") and float(t.vol_ccy_24h or 0) > 10000]
        
        advancing = []
        declining = []
        unchanged = []
        
        for ticker in usdt_tickers:
            change = float(ticker.change_24h or 0)
            if change > 0.001:
                advancing.append(ticker)
            elif change < -0.001:
                declining.append(ticker)
            else:
                unchanged.append(ticker)
        
        print("Market Breadth Analysis (USDT pairs, vol > $10k):")
        print("=" * 60)
        print(f"  Total:      {len(usdt_tickers)} pairs")
        print(f"  Advancing:  {len(advancing)} ({len(advancing)/len(usdt_tickers)*100:.1f}%)")
        print(f"  Declining:  {len(declining)} ({len(declining)/len(usdt_tickers)*100:.1f}%)")
        print(f"  Unchanged:  {len(unchanged)} ({len(unchanged)/len(usdt_tickers)*100:.1f}%)")
        print()
        
        # Top gainers
        top_gainers = sorted(advancing, key=lambda t: float(t.change_24h or 0), reverse=True)[:5]
        print("Top 5 Gainers:")
        for t in top_gainers:
            print(f"  {t.inst_id:<12} {float(t.change_24h)*100:>+7.2f}%  ${float(t.last):>12,.4f}")
        
        # Top losers
        top_losers = sorted(declining, key=lambda t: float(t.change_24h or 0))[:5]
        print("\nTop 5 Losers:")
        for t in top_losers:
            print(f"  {t.inst_id:<12} {float(t.change_24h)*100:>+7.2f}%  ${float(t.last):>12,.4f}")
        
        return usdt_tickers

breadth = await market_breadth()

## Part 5: Real-Time Streaming (WebSocket)

### Stream Ticker Updates

In [ ]:
async def stream_ticker(inst_id: str, duration: int = 10):
    """Stream real-time ticker updates."""
    from okx_client_gw.adapters.websocket import OkxWebSocketClient, OkxWsConfig
    
    ws_config = OkxWsConfig(demo=True)
    service = StreamingService(OkxWebSocketClient(ws_config))
    
    print(f"Streaming {inst_id} ticker for {duration} seconds...")
    print(f"{'Time':>12} {'Price':>12} {'Bid':>12} {'Ask':>12} {'Volume':>15}")
    print("-" * 65)
    
    count = 0
    async for ticker in service.stream_ticker(inst_id):
        count += 1
        print(f"{datetime.now().strftime('%H:%M:%S'):>12} ${float(ticker.last):>10,.2f} ${float(ticker.bid_px):>10,.2f} ${float(ticker.ask_px):>10,.2f} {float(ticker.vol_24h):>15,.2f}")
        
        if count >= duration:  # Limit to duration messages
            break
    
    print(f"\nReceived {count} updates")

# Uncomment to run (will stream for ~10 seconds):
# await stream_ticker("BTC-USDT", duration=10)

### Stream Order Book Updates

In [ ]:
async def stream_orderbook(inst_id: str, duration: int = 5):
    """Stream real-time order book updates."""
    from okx_client_gw.adapters.websocket import OkxWebSocketClient, OkxWsConfig
    
    ws_config = OkxWsConfig(demo=True)
    service = StreamingService(OkxWebSocketClient(ws_config))
    
    print(f"Streaming {inst_id} order book for {duration} updates...")
    print()
    
    count = 0
    async for orderbook in service.stream_orderbook(inst_id, depth=5):
        count += 1
        print(f"Update #{count} at {orderbook.ts.strftime('%H:%M:%S.%f')[:-3]}")
        print(f"  Spread: ${orderbook.spread:.2f}")
        print(f"  Best Bid: ${float(orderbook.bids[0].px):,.2f} x {float(orderbook.bids[0].sz):.4f}")
        print(f"  Best Ask: ${float(orderbook.asks[0].px):,.2f} x {float(orderbook.asks[0].sz):.4f}")
        print()
        
        if count >= duration:
            break
    
    print(f"Received {count} order book updates")

# Uncomment to run:
# await stream_orderbook("BTC-USDT", duration=5)

### Stream Trades

In [ ]:
async def stream_trades(inst_id: str, duration: int = 20):
    """Stream real-time trade updates."""
    from okx_client_gw.adapters.websocket import OkxWebSocketClient, OkxWsConfig
    
    ws_config = OkxWsConfig(demo=True)
    service = StreamingService(OkxWebSocketClient(ws_config))
    
    print(f"Streaming {inst_id} trades...")
    print(f"{'Time':>15} {'Side':>6} {'Price':>12} {'Size':>12}")
    print("-" * 50)
    
    count = 0
    async for trade in service.stream_trades(inst_id):
        count += 1
        side_symbol = "↑ BUY" if trade.side == "buy" else "↓ SELL"
        print(f"{trade.ts.strftime('%H:%M:%S.%f')[:-3]:>15} {side_symbol:>6} ${float(trade.px):>10,.2f} {float(trade.sz):>12.6f}")
        
        if count >= duration:
            break
    
    print(f"\nReceived {count} trades")

# Uncomment to run:
# await stream_trades("BTC-USDT", duration=20)

### Multi-Channel Streaming

In [ ]:
async def multi_stream():
    """Stream multiple instruments simultaneously."""
    from okx_client_gw.adapters.websocket import OkxWebSocketClient, OkxWsConfig
    
    ws_config = OkxWsConfig(demo=True)
    service = MultiChannelStreamingService(OkxWebSocketClient(ws_config))
    
    instruments = ["BTC-USDT", "ETH-USDT", "SOL-USDT"]
    
    print(f"Streaming tickers for: {', '.join(instruments)}")
    print(f"{'Time':>12} {'Instrument':<12} {'Price':>12} {'Change':>10}")
    print("-" * 50)
    
    count = 0
    async for ticker in service.stream_multiple_tickers(instruments):
        count += 1
        change = float(ticker.change_24h or 0) * 100
        print(f"{datetime.now().strftime('%H:%M:%S'):>12} {ticker.inst_id:<12} ${float(ticker.last):>10,.2f} {change:>+9.2f}%")
        
        if count >= 15:  # Show 15 updates
            break
    
    print(f"\nReceived {count} updates across {len(instruments)} instruments")

# Uncomment to run:
# await multi_stream()

## Summary

This notebook demonstrated the market data capabilities of `okx-client-gw-py`:

**Instrument Discovery**:
- Explore market structure (spot, swap, futures, options)
- Find and filter instruments

**Real-Time Data**:
- Market snapshots with tickers, order books, and trades
- Order book depth analysis

**Historical Data**:
- Candlestick (OHLCV) data
- Trade flow analysis

**Multi-Market Analysis**:
- Spot vs swap price comparison (basis trading)
- Market breadth indicators

**Streaming (WebSocket)**:
- Real-time ticker streams
- Order book updates
- Trade stream
- Multi-instrument streaming

All examples use public endpoints and don't require authentication, making them suitable for market research and analysis without trading access.